In [17]:
# Importación de librerias
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

In [5]:
# Cargar el archivo limpio
file_path = '../data/gold/ExporteCOL2022_2023_2024_top_products.csv'
df = pd.read_csv(file_path)

# Mostrar las primeras filas para revisar la estructura
print(df.head())

# Resumen estadístico de las variables numéricas
print(df.describe())

    Año  Mes         Uen               Regional  \
0  2022    1  DECORATIVO  REGIONAL BARRANQUILLA   
1  2022    1  DECORATIVO  REGIONAL BARRANQUILLA   
2  2022    1  DECORATIVO  REGIONAL BARRANQUILLA   
3  2022    1  DECORATIVO  REGIONAL BARRANQUILLA   
4  2022    1  DECORATIVO  REGIONAL BARRANQUILLA   

                 Canal Comercial  Marquilla  Código Producto  \
0  Cadenas y Grandes Superficies  AEROCOLOR         10283186   
1  Cadenas y Grandes Superficies  AEROCOLOR         10283188   
2  Cadenas y Grandes Superficies  AEROCOLOR         10283224   
3  Cadenas y Grandes Superficies  AEROCOLOR         10283229   
4  Cadenas y Grandes Superficies  AEROCOLOR         10283298   

                                           Producto  Numérica Clientes  \
0  LACA BLANCO BRILLANTE AEROCOLOR AEROSOLES 300 ML                  2   
1        LACA NEGRO MATE AEROCOLOR AEROSOLES 300 ML                  1   
2  PINTURA METALICA ORO 18 KILATES AEROCOLOR 300 ML                  2   
3         AL

In [7]:
# Identificar las columnas categóricas
categorical_columns = ['Uen', 'Regional', 'Canal Comercial', 'Marquilla']

# Calcular la cantidad de valores únicos en cada columna categórica
unique_values = {col: df[col].nunique() for col in categorical_columns}

# Mostrar los resultados
print("Cantidad de valores únicos en cada columna categórica:")
for col, count in unique_values.items():
    print(f"{col}: {count} valores únicos")

Cantidad de valores únicos en cada columna categórica:
Uen: 2 valores únicos
Regional: 7 valores únicos
Canal Comercial: 8 valores únicos
Marquilla: 56 valores únicos


In [21]:
# Crear una copia del DataFrame original
df_copy = df.copy()

# Realizar las transformaciones en la copia
# Convertir 'Código Producto' a tipo string
df_copy['Código Producto'] = df_copy['Código Producto'].astype(str)

# Limpiar columnas numéricas
for column in ['Ventas Galones', 'Ventas', 'Utilidad Bruta', 'Costos', 'Margen']:
    if column in df_copy.columns:
        df_copy[column] = df_copy[column].astype(str).str.replace(',', '.').str.strip()
        df_copy[column] = pd.to_numeric(df_copy[column], errors='coerce')

# Excluir filas con margen negativo y otros criterios de limpieza
df_copy = df_copy[df_copy['Margen'] >= 0]
df_copy = df_copy.replace("#DIV/0!", float('nan')).dropna()
df_copy = df_copy[~df_copy['Producto'].str.contains('KIT', case=False, na=False)]
df_copy = df_copy[df_copy['Ventas Galones'] >= 0]
df_copy = df_copy[df_copy['Ventas'] >= 0]
df_copy = df_copy[df_copy['Utilidad Bruta'] >= 0]
df_copy = df_copy[df_copy['Costos'] >= 0]

# Filtrar productos con más de una venta
ventas_por_producto = df_copy.groupby('Código Producto')['Ventas Galones'].count()
productos_una_venta = ventas_por_producto[ventas_por_producto == 1].index
df_copy = df_copy[~df_copy['Código Producto'].isin(productos_una_venta)]

# Codificación de variables categóricas
# One-Hot Encoding para variables con pocos valores únicos
df_encoded = pd.get_dummies(df_copy, columns=['Uen', 'Regional', 'Canal Comercial'], drop_first=True)

# Codificación de 'Marquilla' con LabelEncoder
label_encoder = LabelEncoder()
df_encoded['Marquilla'] = label_encoder.fit_transform(df_encoded['Marquilla'])

# Verificar el resultado final
print(df_encoded.info())
print(df_encoded.head())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 93151 entries, 0 to 93150
Data columns (total 26 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   Año                                    93151 non-null  int64  
 1   Mes                                    93151 non-null  int64  
 2   Marquilla                              93151 non-null  int64  
 3   Código Producto                        93151 non-null  object 
 4   Producto                               93151 non-null  object 
 5   Numérica Clientes                      93151 non-null  int64  
 6   Numérica Documentos                    93151 non-null  int64  
 7   Ventas Galones                         93151 non-null  float64
 8   Ventas                                 93151 non-null  float64
 9   Utilidad Bruta                         93151 non-null  float64
 10  Costos                                 93151 non-null  float64
 11  Ma